In [2]:
import numpy as np
from scipy.special import jv, yv
import pandas as pd

# LSRK5 係數 (Carpenter & Kennedy 1994)
A = np.array([0.0,
              -567301805773.0 / 1357537059087.0,
              -2404267990393.0 / 2016746695238.0,
              -3550918686646.0 / 2091501179385.0,
              -1275806237668.0 / 842570457699.0])

B = np.array([1432997174477.0 / 9575080441755.0,
              5161836677717.0 / 13612068292357.0,
              1720146321549.0 / 2090206949498.0,
              3134564353537.0 / 4481467310338.0,
              2277821191437.0 / 14882151754819.0])

C = np.array([0.0, 0.1496590219993, 0.3704009573644, 0.6222557631345, 0.9582821306748])

# 問題參數
n = 8
c = 1.0
alpha = 6.1556542986056666
omega = alpha * c

In [3]:
def exact_solution(r, theta, t, n, alpha, omega):
    """解析解"""
    A_coef = 1.0
    B_coef = -A_coef * jv(n, alpha * 1) / yv(n, alpha * 1)
    hat_u = A_coef * jv(n, alpha * r) + B_coef * yv(n, alpha * r)
    u = hat_u * np.cos(n * theta - omega * t)
    u_t = omega * hat_u * np.sin(n * theta - omega * t)
    return u, u_t

def construct_D_matrix(N):
    """構建偽譜微分矩陣"""
    D = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            if i != j:
                diff = i - j
                D[i, j] = ((-1)**diff) / (2 * np.tan(np.pi * diff / N))
    return D

def f_theta_numerical(u, R_inv2, N, D2):
    """
    角度項數值計算
    Args:
        u: 解矩陣，形狀 (K+1, N)
        R_inv2: 對角矩陣 diag(1 / r_k^2)，形狀 (K+1, K+1)
        N: 角度格點數
        D2: 二階微分矩陣 (N, N)
    Returns:
        f_theta: 角度項，形狀 (K+1, N)
    """
    result = R_inv2 @ u @ D2  # 矩陣乘法，形狀 (K+1, N)
    return result

def precompute_radial_derivatives(r_arr, Theta, n, alpha):
    """預計算所有半網格點的徑向導數係數（時間無關部分）"""
    K = len(r_arr) - 1  # 內部網格點數
    N = Theta.shape[1]
    
    # 半網格點位置
    r_half = np.zeros(K)
    for k in range(K):
        r_half[k] = (r_arr[k] + r_arr[k+1]) / 2
    
    # 預計算徑向導數的空間部分（時間無關）
    A_coef = 1.0
    B_coef = -A_coef * jv(n, alpha * 1) / yv(n, alpha * 1)
    
    radial_coeff = np.zeros(K)  # 只需要 K 個值，因為對所有角度點都相同
    for k in range(K):
        r_val = r_half[k]
        # 計算 Bessel 函數的導數
        djv_dr = alpha * (jv(n-1, alpha * r_val) - jv(n+1, alpha * r_val)) / 2
        dyv_dr = alpha * (yv(n-1, alpha * r_val) - yv(n+1, alpha * r_val)) / 2
        hat_u_r = A_coef * djv_dr + B_coef * dyv_dr
        
        radial_coeff[k] = r_val * hat_u_r
    
    return radial_coeff

def compute_ru_r_half_grid_optimized(radial_coeff, Theta, t, n, omega):
    """使用預計算的係數快速計算半網格點上的 ru_r（矩陣加速版）"""
    cos_term = np.cos(n * Theta[0, :] - omega * t)  # shape: (N,)
    
    # 向量化：radial_coeff shape (K,) -> (K, 1), cos_term shape (N,) -> (1, N)
    ru_r_half = radial_coeff[:, np.newaxis] * cos_term[np.newaxis, :]  # shape: (K, N)
    
    return ru_r_half


def f_r_numerical_optimized(u, r_arr, radial_coeff, Theta, t, n, omega):
    """徑向項數值計算 - 矩陣加速版本，使用預計算的徑向導數"""
    K = len(r_arr) - 1
    dr = 1.0 / K
    N = u.shape[1]
    
    # 使用預計算的係數快速計算半網格點上的 ru_r
    ru_r_half = compute_ru_r_half_grid_optimized(radial_coeff, Theta, t, n, omega)  # shape = (K, N)

    # 計算內部點的徑向項：向量化處理
    r_k = r_arr[1:K].reshape(-1, 1)  # shape = (K-1, 1)
    numerator = ru_r_half[1:K, :] - ru_r_half[0:K-1, :]  # shape = (K-1, N)
    f_r_inner = numerator / (r_k * dr)  # shape = (K-1, N)

    # 初始化結果陣列
    f_r = np.zeros_like(u)
    f_r[1:K, :] = f_r_inner

    # 邊界點 k=0 和 k=K 已經是 0，不需要額外設定（除非你要明確指定）
    f_r[0, :] = 0.0
    f_r[K, :] = 0.0

    return f_r


def enforce_boundary_conditions(u, v):
    """強加 Dirichlet 邊界條件 u=0"""
    u[0, :] = 0.0   # r=1 邊界
    u[-1, :] = 0.0  # r=2 邊界
    v[0, :] = 0.0   # r=1 邊界
    v[-1, :] = 0.0  # r=2 邊界

def rhs_system_part4_optimized(u, v, radial_coeff, r_arr, N, Theta, t, n, omega, D2, R_inv2):
    """系統右端項 - Part IV 優化版本"""
    du_dt = v
    dv_dt = f_r_numerical_optimized(u, r_arr, radial_coeff, Theta, t, n, omega) + f_theta_numerical(u, R_inv2, N, D2)
    return du_dt, dv_dt

In [4]:
# 有限差分方法徑向離散化 - 簡化版本
N = 192  # 固定角度網格點數
CFL = 0.5
T = 2 * np.pi / alpha
K_list = [20, 40, 80, 160]
errors = []

for K in K_list:
    D = construct_D_matrix(N)
    D2 = D.T @ D.T
    
    # 網格設置 - 改為 (K+1, N) 形狀
    r = np.linspace(1, 2, K + 1)
    R_inv2 = np.diag(1 / r**2)  # (K+1, K+1)
    theta = np.linspace(0, 2 * np.pi, N, endpoint=False)
    dr = 1 / K
    dtheta = 2 * np.pi / N
    R, Theta = np.meshgrid(r, theta, indexing='ij')  # 改變 indexing 為 'ij'
    
    # 預計算徑向導數係數（只需計算一次）
    radial_coeff = precompute_radial_derivatives(r, Theta, n, alpha)
    
    # 時間步進
    dt = CFL * min(dtheta, dr)
    num_steps = int(np.ceil(T / dt))
    dt = T / num_steps
    
    # 初始條件
    u, v = exact_solution(R, Theta, 0, n, alpha, omega)
    
    # 強加初始邊界條件
    enforce_boundary_conditions(u, v)
    
    # LSRK5 時間步進
    for step in range(num_steps):
        t_n = step * dt
        u_stage, v_stage = u.copy(), v.copy()
        du, dv = np.zeros_like(u), np.zeros_like(v)
        
        for j in range(5):
            t_stage = t_n + C[j] * dt
            du_dt, dv_dt = rhs_system_part4_optimized(u_stage, v_stage, radial_coeff, r, N, Theta, t_stage, n, omega, D2, R_inv2)
            du = A[j] * du + dt * du_dt
            dv = A[j] * dv + dt * dv_dt
            u_stage += B[j] * du
            v_stage += B[j] * dv
            
            # 每個階段後強加邊界條件
            enforce_boundary_conditions(u_stage, v_stage)
        
        u, v = u_stage, v_stage
    
    # 誤差計算
    u_exact, _ = exact_solution(R, Theta, T, n, alpha, omega)
    max_error = np.max(np.abs(u - u_exact))
    errors.append(max_error)

# 收斂率計算
rates = [np.nan]
for i in range(1, len(errors)):
    rate = np.log(errors[i-1] / errors[i]) / np.log(K_list[i] / K_list[i-1])
    rates.append(rate)

# 結果表格
df = pd.DataFrame({
    "(K, N)": [f"({K}, {N})" for K in K_list],
    "Error": errors,
    "Rate": rates
})

print("數值結果:")
print(df.to_string(index=False, float_format="%.6e"))

數值結果:
    (K, N)        Error         Rate
 (20, 192) 1.284472e-03          NaN
 (40, 192) 3.213667e-04 1.998883e+00
 (80, 192) 8.033892e-05 2.000050e+00
(160, 192) 2.008658e-05 1.999867e+00
